In [1]:
# is base model we have here good enough? - we ended up using albert
# explore improvements
  # Maybe different flavor of bert? [X]
    # * fine tuned albert
  # Maybe try a few different models and measure F1 scores? [X]
    # * this fine tuned albert has F1 score of 87.4628460501696 which is very good
  # Maybe try and fine tune on a different dataset? [X]
    # * SQUAD 2.0 seems to be good enough
  # better information retrieval? [X]
    # providing good context, maybe a simple ML algo to provide good context?
    # * we're just using cosine similarity of tf-idf vector space and it seems to work fine
  # Chunk sentances better ?
    # we need to chunk things better. Not sure how yet.
    
  # a few ideas that can help with first three improvemetns are here https://towardsdatascience.com/which-flavor-of-bert-should-you-use-for-your-qa-task-6d6a0897fb24

In [2]:
import torch
import wikipedia
import tensorflow as tf
import numpy as np

from collections import OrderedDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TFAutoModelForQuestionAnswering

In [3]:
model = "ktrapeznikov/albert-xlarge-v2-squad-v2"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model, from_pt=True)

All PyTorch model weights were used when initializing TFAlbertForQuestionAnswering.

All the weights of TFAlbertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFAlbertForQuestionAnswering for predictions without further training.


In [24]:
question = "What is the population of Columbus, Ohio?"

q_search = wikipedia.search(question)
page = wikipedia.page(q_search[0])
#page = wikipedia.page(q_search[2])
context = page.content

In [25]:
q_search

['Columbus, Ohio',
 'Christopher Columbus',
 'List of United States cities by population',
 'Ohio',
 'Ohio Penitentiary',
 'Neighborhoods in Columbus, Ohio',
 'Dublin, Ohio',
 'Columbus City Center',
 'Westerville, Ohio',
 "Ohio's 4th congressional district"]

In [26]:
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="tf")

Token indices sequence length is longer than the specified maximum sequence length for this model (9 > 512). Running this sequence through the model will result in indexing errors


In [27]:
max_length = model.config.max_position_embeddings

In [28]:
question_mask = inputs["token_type_ids"] == 0

In [29]:
question_tensor = tf.boolean_mask(inputs["input_ids"], question_mask)

In [30]:
chunk_size = max_length - len(question_tensor) - 1

In [31]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [32]:
chunked_input = OrderedDict()

In [33]:
for k, v in inputs.items():
    question = tf.boolean_mask(v, question_mask)
    context =  tf.boolean_mask(v, ~question_mask)
    chunks = list(get_chunks(context, chunk_size))
    
    for i, chunk in enumerate(chunks):
        if i not in chunked_input:
            chunked_input[i] = {}
            
        entire_input = tf.concat([question, chunk], axis=0)
        
        if i != len(chunks) - 1:
            if k == 'input_ids':
                entire_input = tf.concat([entire_input, tf.constant([102])], axis=0)
            else:
                entire_input = tf.concat([entire_input, tf.constant([1])], axis=0)
        
        chunked_input[i][k] = tf.reshape(entire_input, [1, entire_input.shape[0]])

In [34]:
chunked_input

OrderedDict([(0,
              {'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
               array([[    2,    98,    25,    14,   350,    16,  8950,    15,  2393,
                          60,     3,  8950,    25,    14,   146,  1057,    17,    14,
                         127, 23464,   136,    19,    14,   287,     9,    18,     9,
                         146,    16,  2393,     9,    29,    21,   350,    16,   469,
                        3804,    15,  4083,   240,    28,    16, 10114, 10527,    15,
                          32,    25,    14,   513,    96,     8,  4630, 23464,   136,
                          19,    14,   181,   202,    15,    14,   153,     8,  4630,
                       23464,   136,    19,    14, 15727,    75,  1360,    15,  2695,
                          15,    17,    14,   422,     8,  4630, 23464,   146,  1057,
                          75,  6014,    15,  4045,    17,  4665,    15,  1338,     9,
                          32,    25,    14,  28

In [52]:
answer = ''
probs = []

for k, chunk in chunked_input.items():
    start_scores, end_scores = model(chunk)

    answer_start = tf.argmax(start_scores, axis=1).numpy()[0]
    answer_end = (tf.argmax(end_scores, axis=1) + 1).numpy()[0]
    answ = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(chunk['input_ids'][0][answer_start:answer_end]))
    
    if answ != '' and answ != '[CLS]':
        probs.append([np.take(start_scores.numpy(), answer_start, axis=1)[0], np.take(end_scores.numpy(), answer_end, axis=1)[0]])
    
    if answ != '[CLS]':
        answer += answ + " / "

In [53]:
answer

'898,553 / 3,500 / 787,033 / '

In [55]:
probs

[[5.5390215, -5.4146104], [6.162928, 2.0516958], [5.3882284, 2.2797961]]

In [1]:
# Evalauting on dev set and making better preds

In [6]:
from transformers.data.processors.squad import SquadV2Processor

In [7]:
processor = SquadV2Processor()
examples = processor.get_dev_examples("dataset/", filename="dev-v2.0.json")

100%|██████████| 35/35 [00:02<00:00, 11.88it/s]


In [8]:
len(examples) # ~12k in dev set

11873

In [9]:
question_id_example_index_mapping = {example.qas_id: i for i, example in enumerate(examples)}

In [10]:
question_id_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}

In [11]:
answer_question_ids = [question_id for question_id, has_answer in question_id_to_has_answer.items() if has_answer]

In [12]:
no_answer_question_ids = [question_id for question_id, has_answer in question_id_to_has_answer.items() if not has_answer]

In [13]:
question_id_example_index_mapping[answer_question_ids[0]]

0

In [14]:
examples[0].question_text

'In what country is Normandy located?'

In [15]:
examples[0].context_text

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [16]:
examples[0].answers

[{'text': 'France', 'answer_start': 159},
 {'text': 'France', 'answer_start': 159},
 {'text': 'France', 'answer_start': 159},
 {'text': 'France', 'answer_start': 159}]

In [17]:
def get_pred(question_id):
    question = examples[question_id_example_index_mapping[question_id]].question_text
    context = examples[question_id_example_index_mapping[question_id]].context_text
    
    inputs = tokenizer.encode_plus(question, context, return_tensors='tf')
    
    start_scores, end_scores = model(inputs)
    
    answer_start = tf.argmax(start_scores, axis=1).numpy()[0]
    answer_end = (tf.argmax(end_scores, axis=1) + 1).numpy()[0]
    
    answer =  tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    
    return answer

In [18]:
# bunch of helper functions from squad_metrics.py by HF

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [19]:
prediction = get_pred(answer_question_ids[1300])
example = examples[question_id_example_index_mapping[answer_question_ids[1300]]]

gold_answers = get_gold_answers(example)

f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)
exact_match_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)

In [20]:
f1_score, exact_match_score

(0.8, 0)

In [21]:
prediction

'water bodies'

In [22]:
gold_answers

['water',
 "in solution in the world's water bodies",
 "the world's water bodies"]

In [23]:
prediction = get_pred(no_answer_question_ids[1254])
example = examples[question_id_example_index_mapping[no_answer_question_ids[1254]]]

gold_answers = get_gold_answers(example)

f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)
exact_match_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)

In [24]:
f1_score, exact_match_score

(0, 0)

In [3]:
!python run_squad.py  \
    --model_type albert   \
    --model_name_or_path ktrapeznikov/albert-xlarge-v2-squad-v2  \
    --output_dir models/albert-xlarge-v2-squad-v2 \
    --data_dir dataset   \
    --predict_file dev-v2.0.json   \
    --do_eval   \
    --version_2_with_negative \
    --do_lower_case  \
    --per_gpu_eval_batch_size 2   \
    --max_seq_length 384   \
    --doc_stride 128

2020-08-19 06:35:31.510311: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
08/19/2020 06:35:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/19/2020 06:35:32 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/ktrapeznikov/albert-xlarge-v2-squad-v2/config.json from cache at /home/novarac23/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f
08/19/2020 06:35:32 - INFO - transformers.configuration_utils -   Model config AlbertConfig {
  "architectures": [
    "AlbertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size":

08/19/2020 06:35:38 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/ktrapeznikov/albert-xlarge-v2-squad-v2/pytorch_model.bin from cache at /home/novarac23/.cache/torch/transformers/49e7ccd95cebd1b8210607e08d6d9ad7dccd760db94a9a409c19a93f9ec5594c.e63d894d1c8b12d9da1be391592635fa9414da143b07cc6d63895911e2df8fb2
08/19/2020 06:35:39 - INFO - transformers.modeling_utils -   All model checkpoint weights were used when initializing AlbertForQuestionAnswering.

08/19/2020 06:35:39 - INFO - transformers.modeling_utils -   All the weights of AlbertForQuestionAnswering were initialized from the model checkpoint at ktrapeznikov/albert-xlarge-v2-squad-v2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use AlbertForQuestionAnswering for predictions without further training.
08/19/2020 06:35:39 - INFO - __main__ -   Creating features from dataset file at dataset
100%|███████████████████████████████████████████| 

In [4]:
results = {'exact': 84.41842836688285, 'f1': 87.4628460501696, 'total': 11873, 'HasAns_exact': 80.68488529014844, 'HasAns_f1': 86.78245127423482, 'HasAns_total': 5928, 'NoAns_exact': 88.1412952060555, 'NoAns_f1': 88.1412952060555, 'NoAns_total': 5945, 'best_exact': 84.41842836688285, 'best_exact_thresh': 0.0, 'best_f1': 87.46284605016956, 'best_f1_thresh': 0.0}

In [6]:
results

{'exact': 84.41842836688285,
 'f1': 87.4628460501696,
 'total': 11873,
 'HasAns_exact': 80.68488529014844,
 'HasAns_f1': 86.78245127423482,
 'HasAns_total': 5928,
 'NoAns_exact': 88.1412952060555,
 'NoAns_f1': 88.1412952060555,
 'NoAns_total': 5945,
 'best_exact': 84.41842836688285,
 'best_exact_thresh': 0.0,
 'best_f1': 87.46284605016956,
 'best_f1_thresh': 0.0}

In [26]:
inputs = tokenizer.encode_plus(example.question_text, example.context_text, add_special_tokens=True, return_special_tokens_mask=True, return_tensors='tf')

In [57]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy()[0]))

"[CLS] what happened 3.7-2 billion years ago?[SEP] free oxygen gas was almost nonexistent in earth's atmosphere before photosynthetic archaea and bacteria evolved, probably about 3.5 billion years ago. free oxygen first appeared in significant quantities during the paleoproterozoic eon (between 3.0 and 2.3 billion years ago). for the first billion years, any free oxygen produced by these organisms combined with dissolved iron in the oceans to form banded iron formations. when such oxygen sinks became saturated, free oxygen began to outgas from the oceans 3–2.7 billion years ago, reaching 10% of its present level around 1.7 billion years ago.[SEP]"

In [29]:
inputs['input_ids'][0][:13]

<tf.Tensor: shape=(13,), dtype=int32, numpy=
array([   2,   98, 1190,  203,    9,  465,    8,  135, 2786,  122, 1464,
         60,    3], dtype=int32)>

In [98]:
inputs['input_ids'][-1]

3

In [30]:
start_scores, end_scores = model(inputs)

In [31]:
start_scores_list = start_scores.numpy()[0]
end_scores_list = end_scores.numpy()[0]

In [32]:
start_index_and_score = sorted(enumerate(start_scores_list), key=lambda x: x[1], reverse=True)
end_index_and_score = sorted(enumerate(end_scores_list), key=lambda x: x[1], reverse=True)

In [33]:
start_index_and_score[:2], end_index_and_score[:2]

([(110, 1.7505212), (0, 0.4204398)], [(115, 0.8846194), (119, 0.70913386)])

In [34]:
# we're just going to take top 5 predictions

start_indexes = [idx for idx, score in start_index_and_score[:5]]
end_indexes = [idx for idx, score in end_index_and_score[:5]]

In [39]:
inputs['input_ids'][0].numpy()

array([    2,    98,  1190,   203,     9,   465,     8,   135,  2786,
         122,  1464,    60,     3,   551,  6838,  1692,    23,   557,
         538, 20788,    19,  1073,    22,    18,  6341,   115,  3056,
       29772,  5071, 17854,    17, 10955,  7339,    15,   910,    88,
         203,     9,   264,  2786,   122,  1464,     9,   551,  6838,
          64,   682,    19,  1505, 13787,   112,    14, 17038, 17588,
         661,  2500,   596,    13, 11572,    13,     5, 14427,   203,
           9,   387,    17,   172,     9,   240,  2786,   122,  1464,
           6,     9,    26,    14,    64,  2786,   122,    15,   186,
         551,  6838,   671,    34,   158, 13039,  2461,    29,  7895,
        1751,    19,    14,  2144,    18,    20,   505,   323,    69,
        1751, 16798,     9,    76,   145,  6838,  6046,    18,   178,
       26833,    15,   551,  6838,   260,    20,    70,  6927,    37,
          14,  2144,    18,   203,    10,   135,     9,   465,  2786,
         122,  1464,

In [41]:
# get indexes of question token ids 

tokens = list(inputs['input_ids'][0].numpy())

question_indexes = [i+1 for i, token in enumerate(tokens[1:tokens.index(3)])]

In [42]:
question_indexes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [47]:
# start collecting preliminary answers based on sensible solutions

import collections

preliminary_prediction = collections.namedtuple("preliminary_prediction", ["start_index", "end_index", "start_score", "end_score"])

In [53]:
preliminary_preds = []

for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index in question_indexes:
            continue
            
        if end_index in question_indexes:
            continue
            
        if end_index < start_index:
            continue
            
        preliminary_preds.append(preliminary_prediction(start_index=start_index, end_index=end_index, start_score=start_scores.numpy()[0][start_index], end_score=end_scores.numpy()[0][end_index]))

In [54]:
preliminary_preds

[preliminary_prediction(start_index=110, end_index=115, start_score=1.7505212, end_score=0.8846194),
 preliminary_prediction(start_index=110, end_index=119, start_score=1.7505212, end_score=0.70913386),
 preliminary_prediction(start_index=110, end_index=127, start_score=1.7505212, end_score=-2.6045392),
 preliminary_prediction(start_index=110, end_index=126, start_score=1.7505212, end_score=-3.6000865),
 preliminary_prediction(start_index=0, end_index=115, start_score=0.4204398, end_score=0.8846194),
 preliminary_prediction(start_index=0, end_index=119, start_score=0.4204398, end_score=0.70913386),
 preliminary_prediction(start_index=0, end_index=0, start_score=0.4204398, end_score=0.19268568),
 preliminary_prediction(start_index=0, end_index=127, start_score=0.4204398, end_score=-2.6045392),
 preliminary_prediction(start_index=0, end_index=126, start_score=0.4204398, end_score=-3.6000865),
 preliminary_prediction(start_index=111, end_index=115, start_score=-1.3322576, end_score=0.8846

In [55]:
# sort prelim preds by their score

prelim_preds = sorted(preliminary_preds, key=lambda x: (x.start_score + x.end_score), reverse=True)

In [56]:
prelim_preds[:5]

[preliminary_prediction(start_index=110, end_index=115, start_score=1.7505212, end_score=0.8846194),
 preliminary_prediction(start_index=110, end_index=119, start_score=1.7505212, end_score=0.70913386),
 preliminary_prediction(start_index=0, end_index=115, start_score=0.4204398, end_score=0.8846194),
 preliminary_prediction(start_index=0, end_index=119, start_score=0.4204398, end_score=0.70913386),
 preliminary_prediction(start_index=0, end_index=0, start_score=0.4204398, end_score=0.19268568)]

In [64]:
# convert to text and get top 5 five predictions

best_prediction = collections.namedtuple("best_prediction", ["text", "start_score", "end_score"])

nbest = []
seen_predictions = []

for pred in prelim_preds:
    if len(nbest) >= 5:
        break
    
    if pred.start_index > 0:
        text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens[pred.start_index:pred.end_index+1]))
        text = text.strip()
        text = " ".join(text.split())
        
    if text in seen_predictions:
        continue
    
    seen_predictions.append(text)
    nbest.append(best_prediction(text=text, start_score=pred.start_score, end_score=pred.end_score))
    
nbest.append(best_prediction(text="", start_score=start_scores.numpy()[0][0], end_score=end_scores.numpy()[0][0]))

In [65]:
nbest

[best_prediction(text='free oxygen began to outgas', start_score=1.7505212, end_score=0.8846194),
 best_prediction(text='free oxygen began to outgas from the oceans', start_score=1.7505212, end_score=0.70913386),
 best_prediction(text='oxygen began to outgas', start_score=-1.3322576, end_score=0.8846194),
 best_prediction(text='oxygen began to outgas from the oceans', start_score=-1.3322576, end_score=0.70913386),
 best_prediction(text='free oxygen began to outgas from the oceans 3–2.7 billion years ago', start_score=1.7505212, end_score=-2.6045392),
 best_prediction(text='', start_score=0.4204398, end_score=0.19268568)]

In [66]:
# compute null score 

null_score = start_scores.numpy()[0][0] + end_scores.numpy()[0][0] 

score_diff = null_score - nbest[0].start_score - nbest[0].end_score

In [67]:
score_diff

-2.022015

In [69]:
import json

In [73]:
null_odds = json.load(open('models/albert-xlarge-v2-squad-v2/null_odds_.json', 'rb'))

In [74]:
null_odds[example.qas_id]

-2.022022172808647

In [75]:
preds_og = json.load(open('models/albert-xlarge-v2-squad-v2/predictions_.json', 'rb'))

from transformers.data.metrics.squad_metrics import squad_evaluate

results_default_thresh = squad_evaluate(examples, 
                                        preds_og, 
                                        no_answer_probs=null_odds, 
                                        no_answer_probability_threshold=1.0)

In [76]:
results_default_thresh

OrderedDict([('exact', 84.41842836688285),
             ('f1', 87.4628460501696),
             ('total', 11873),
             ('HasAns_exact', 80.68488529014844),
             ('HasAns_f1', 86.78245127423482),
             ('HasAns_total', 5928),
             ('NoAns_exact', 88.1412952060555),
             ('NoAns_f1', 88.1412952060555),
             ('NoAns_total', 5945),
             ('best_exact', 84.7805946264634),
             ('best_exact_thresh', -1.305284634232521),
             ('best_f1', 87.74837464084158),
             ('best_f1_thresh', -1.1585063934326172)])

In [78]:

best_f1_thresh = -1.1585063934326172
results_f1_thresh = squad_evaluate(examples, 
                                   preds_og, 
                                   no_answer_probs=null_odds, 
                                   no_answer_probability_threshold=best_f1_thresh)

In [79]:
results_f1_thresh

OrderedDict([('exact', 84.7805946264634),
             ('f1', 87.74837464084212),
             ('total', 11873),
             ('HasAns_exact', 79.53778677462888),
             ('HasAns_f1', 85.48185764350849),
             ('HasAns_total', 5928),
             ('NoAns_exact', 90.00841042893188),
             ('NoAns_f1', 90.00841042893188),
             ('NoAns_total', 5945),
             ('best_exact', 84.7805946264634),
             ('best_exact_thresh', -1.305284634232521),
             ('best_f1', 87.74837464084158),
             ('best_f1_thresh', -1.1585063934326172)])

In [1]:
def prediction_probabilities(predictions):

    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    all_scores = [pred.start_score+pred.end_score for pred in predictions] 
    return softmax(np.array(all_scores))

In [2]:
def get_answer(question, context, tokenizer, nbest=5, null_thresh=-1.1585063934326172):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_special_tokens_mask=True, return_tensors='tf')
    start_scores, end_scores = model(inputs)
    
    start_scores_list = start_scores.numpy()[0]
    end_scores_list = end_scores.numpy()[0]
    
    start_index_and_score = sorted(enumerate(start_scores_list), key=lambda x: x[1], reverse=True)
    end_index_and_score = sorted(enumerate(end_scores_list), key=lambda x: x[1], reverse=True)
    
    start_indexes = [idx for idx, score in start_index_and_score[:5]]
    end_indexes = [idx for idx, score in end_index_and_score[:5]]
    
    preliminary_prediction = collections.namedtuple("preliminary_prediction", ["start_index", "end_index", "start_score", "end_score"])
    
    tokens = list(inputs['input_ids'][0].numpy())

    question_indexes = [i+1 for i, token in enumerate(tokens[1:tokens.index(3)])]
    
    preliminary_preds = []

    for start_index in start_indexes:
        for end_index in end_indexes:
            if start_index in question_indexes:
                continue

            if end_index in question_indexes:
                continue

            if end_index < start_index:
                continue

            preliminary_preds.append(preliminary_prediction(start_index=start_index, 
                                                            end_index=end_index, 
                                                            start_score=start_scores.numpy()[0][start_index], 
                                                            end_score=end_scores.numpy()[0][end_index]))
            
    prelim_preds = sorted(preliminary_preds, key=lambda x: (x.start_score + x.end_score), reverse=True)
    
    best_prediction = collections.namedtuple("best_prediction", ["text", "start_score", "end_score"])

    nbest = []
    seen_predictions = []

    for pred in prelim_preds:
        if len(nbest) >= 5:
            break

        if pred.start_index > 0:
            text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens[pred.start_index:pred.end_index+1]))
            text = text.strip()
            text = " ".join(text.split())
        else:
            continue

        if text in seen_predictions:
            continue

        seen_predictions.append(text)
        nbest.append(best_prediction(text=text, start_score=pred.start_score, end_score=pred.end_score))

    nbest.append(best_prediction(text="", start_score=start_scores.numpy()[0][0], end_score=end_scores.numpy()[0][0]))
    
    probabilities = prediction_probabilities(nbest)
    
    null_score = start_scores.numpy()[0][0] + end_scores.numpy()[0][0] 

    score_diff = null_score - nbest[0].start_score - nbest[0].end_score
    
    if score_diff > null_thresh:
        return ""
    else:
        return  nbest[0].text, probabilities[0]

In [177]:
tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")

In [178]:
get_answer(example.question_text, example.context_text, tokenizer)

'free oxygen began to outgas'

In [179]:
example2 = examples[question_id_example_index_mapping[no_answer_question_ids[1234]]]

In [180]:
example2.question_text

'What have scientists proposed using to characterize plant platform status?'

In [181]:
example2.context_text

"Oxygen presents two spectrophotometric absorption bands peaking at the wavelengths 687 and 760 nm. Some remote sensing scientists have proposed using the measurement of the radiance coming from vegetation canopies in those bands to characterize plant health status from a satellite platform. This approach exploits the fact that in those bands it is possible to discriminate the vegetation's reflectance from its fluorescence, which is much weaker. The measurement is technically difficult owing to the low signal-to-noise ratio and the physical structure of vegetation; but it has been proposed as a possible method of monitoring the carbon cycle from satellites on a global scale."

In [182]:
get_answer(example2.question_text, example2.context_text, tokenizer)

''

In [189]:
example3 = examples[question_id_example_index_mapping[answer_question_ids[10]]]

In [190]:
example3.question_text

'When was the Duchy of Normandy founded?'

In [191]:
example3.context_text

'In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal property. The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was situated in the former Frankish kingdom of Neustria. The treaty offered Rollo and his men the French lands between the river Epte and the Atlantic coast in exchange for their protection against further Viking incursions. The area corresponded to the northern part of present-day Upper Normandy down to the river Seine, but the Duchy would eventually extend west beyond the Seine. The territory was roughly equivalent to the old province of Rouen, and reproduced the Roman administrative structure of Gallia Lugdunensis II (part of the former Gallia Lugdunensis).'

In [192]:
get_answer(example3.question_text, example3.context_text, tokenizer)

'911'

In [193]:
example3.qas_id

'56dde0ba66d3e219004dad75'

In [194]:
example3.answers

[{'text': '911', 'answer_start': 244},
 {'text': '911', 'answer_start': 244},
 {'text': '911', 'answer_start': 244}]